<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-the-stocks" data-toc-modified-id="Loading-the-stocks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading the stocks</a></span></li><li><span><a href="#Getting-the-AI-started" data-toc-modified-id="Getting-the-AI-started-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting the AI started</a></span><ul class="toc-item"><li><span><a href="#Regression" data-toc-modified-id="Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Regression</a></span></li></ul></li></ul></div>

# AI Part

In [1]:
# cell for autoreload includes
%load_ext autoreload
%autoreload 2

## Loading the stocks

In [2]:
from src.stocks import Stocks
# The warnings are due to the includes in the file

22/06/20 14:14:04 WARN Utils: Your hostname, Mathieus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.27.99.57 instead (on interface feth4588)
22/06/20 14:14:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/20 14:14:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


ModuleNotFoundError: No module named 'org'

We define the schema of the datas contained in files

In [ ]:
from pyspark.sql.types import DoubleType, StringType, DateType, StructType, StructField

In [ ]:
data_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', DoubleType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])

We load all the stocks:

In [ ]:
stocks = Stocks(header=True, delimiter=',', schema=data_schema)

We show an exemple of stock with all the indicators:

In [ ]:
stocks.stocks[0].df.select('High').first()[0]

In [ ]:
stocks.stocks[0].predict.fullDF.show(4)

## Linear Regression

Using our estimators, we predict the next value of "Close":

In [ ]:
stocks.call_prediction_function("print_linear_regression")

We plot the result:

In [ ]:
stocks.call_prediction_function("plot_linear_regression")

Percent of good prediction of up/down:

In [ ]:
stocks.call_prediction_function("percent_good_up_down")

Plot how much we won/lost:

In [ ]:
stocks.call_prediction_function("plot_gain_predict")

Print how much we made:

In [ ]:
stocks.call_prediction_function("print_stonks_or_not")